# seagliderOG1 demo

The purpose of this notebook is to demonstrate the functionality of `seagliderOG1` to convert from Seaglider basestation files to OG1 format.

- OG1 format is a newly agreed format (since June 2024) for glider data sets from various platforms (e.g., Seaglider, Slocum, Seaexplorer).  It lives on github here: (https://github.com/OceanGlidersCommunity/OG-format-user-manual).
- OG1 manual: https://oceangliderscommunity.github.io/OG-format-user-manual/OG_Format.html

The test case is to convert sg015 data from the Labrador Sea in September 2004.

The demo is organised to show

- Conversion of a single dive cycle (single `p*.nc` file)

- Conversion for a folder of local dive-cycle files (full mission of `p*.nc` files)

- Download from remote server + conversion (directory with full mission of `p*.nc` files)

Options are provided to only load e.g. 10 files, but note that OG1 format expects a full mission.


In [1]:
import pathlib
import sys

script_dir = pathlib.Path().parent.absolute()
parent_dir = script_dir.parents[0]
sys.path.append(str(parent_dir))
sys.path.append(str(parent_dir) + '/seagliderOG1')
print(parent_dir)
print(sys.path)

import xarray as xr
import os
import pooch
from seagliderOG1 import readers, writers, plotters, tools
from seagliderOG1 import convertOG1, vocabularies


/Users/eddifying/Cloudfree/gitlab-cloudfree/seagliderOG1
['/Users/eddifying/micromamba/envs/glidertest_env/lib/python312.zip', '/Users/eddifying/micromamba/envs/glidertest_env/lib/python3.12', '/Users/eddifying/micromamba/envs/glidertest_env/lib/python3.12/lib-dynload', '', '/Users/eddifying/micromamba/envs/glidertest_env/lib/python3.12/site-packages', '__editable__.glidertest-0.0.2.dev69+g7c8b81c.finder.__path_hook__', '/Users/eddifying/Cloudfree/gitlab-cloudfree/seagliderOG1', '/Users/eddifying/Cloudfree/gitlab-cloudfree/seagliderOG1/seagliderOG1']


In [11]:
from seagliderOG1 import vocabularies
uc = vocabularies.unit_conversion

print(uc.keys())

input_unit = 'dbar'



dict_keys(['cm/s', 'cm s-1', 'm/s', 'm s-1', 'S/m', 'S m-1', 'mS/cm', 'mS cm-1', 'dbar', 'Pa', 'degrees_Celsius', 'Celsius', 'm', 'cm', 'km', 'g m-3', 'kg m-3'])
['dbar']


In [3]:
from seagliderOG1 import tools
test_pairs = {
        # 'var_name': ('current_units', 'new_units', 'current_value', 'converted_value')
        'velo1': ('cm/s', 'm/s', 100, 1.0),
        'velo2': ('m/s', 'cm/s', 1.0, 100),
        'velo3': ('cm s-1', 'm s-1', 100, 1.0),
        'conduct1': ('S/m', 'mS/cm', 10, 1.0),
        'conduct2': ('mS/cm', 'S/m', 1.0, 10),
        'pres1': ('dbar', 'Pa', 1, 10000),
        'pres2': ('Pa', 'dbar', 10000, 1),
        'pres3': ('dbar', 'kPa', 1, 10),
        'dist1': ('m', 'cm', 1, 100),
        'dist2': ('m', 'km', 1000, 1.0),
        'dist3': ('cm', 'm', 100, 1.0),
        'dist4': ('km', 'm', 1, 1000),
        'density1': ('g m-3', 'kg m-3', 1000, 1),
        'density2': ('kg m-3', 'g m-3', 1, 1000),
        'temp': ('degrees_Celcius', 'Celcius', 1, 1),
    }
for var_name, (current_units, new_units, current_value, new_value) in test_pairs.items():
    converted_values, new_unit = tools.convert_units_var(current_value, current_units, new_units)
    if converted_values != new_value:
        print(var_name + ':' + str(current_value) + ' ' + current_units +'-> ' + new_units + ': gave ' + str(converted_values) + ' but should be ' + str(new_value))
        if new_unit != new_units:
            print('Units')    

velo3:100 cm s-1-> m s-1: gave 100 but should be 1.0
Units
density1:1.0 g m-3-> kg m-3: gave 1.0 but should be 0.001
Units
density2:1 kg m-3-> g m-3: gave 1 but should be 1000
Units


In [3]:
# Specify the path for writing datafiles
data_path = os.path.join(parent_dir, 'data')

## Reading basestation files

This has three ways to load a glider dataset.

Load an example dataset using `seagliderOG1.fetchers.load_sample_dataset`

Alternatively, use your own with e.g. `ds = xr.open_dataset('/path/to/yourfile.nc')`

### Load single sample dataset

In [4]:
ds = readers.load_sample_dataset()
ds

<xarray.Dataset>
Dimensions:                       (sg_data_point: 1180, trajectory: 1,
                                   gps_info: 3, gc_event: 140)
Coordinates:
  * trajectory                    (trajectory) int32 500
    longitude                     (sg_data_point) float64 ...
    latitude                      (sg_data_point) float64 ...
    ctd_time                      (sg_data_point) datetime64[ns] ...
    ctd_depth                     (sg_data_point) float64 ...
Dimensions without coordinates: sg_data_point, gps_info, gc_event
Data variables: (12/226)
    surface_curr_north            float64 ...
    surface_curr_east             float64 ...
    start_of_climb_time           timedelta64[ns] ...
    sg_cal_volmax                 float64 ...
    sg_cal_vbd_min_cnts           int32 ...
    sg_cal_vbd_max_cnts           int32 ...
    ...                            ...
    conductivity                  (sg_data_point) float64 ...
    buoyancy                      (sg_data_point) float64 ...
    GPSE_qc                       |S1 ...
    GPS2_qc                       |S1 ...
    GPS1_qc                       |S1 ...
    CTD_qc                        |S1 ...
Attributes: (12/58)
    quality_control_version:         1.1
    base_station_micro_version:      3705
    time_coverage_resolution:        PT1S
    geospatial_vertical_max:         989.6984588132262
    sea_name:                        Labrador Sea
    mission:                         1
    ...                              ...
    disclaimer:                      Data has not been reviewed and is provid...
    geospatial_vertical_positive:    no
    date_created:                    2013-08-01T05:08:54Z
    geospatial_vertical_units:       meter
    dive_number:                     500
    history:                         Processing start:\n05:52:52 01 Aug 2013 ...

### Load datasets from a local directory

In [5]:
# Specify the input directory on your local machine
input_dir = "/Users/eddifying/Dropbox/data/sg015-ncei-download"

# Load and concatenate all datasets in the input directory
# Optionally, specify the range of profiles to load (start_profile, end_profile)
list_datasets = readers.read_basestation(input_dir, start_profile=500, end_profile=503)

# Where list_datasets is a list of xarray datasets.  A single dataset can be accessed as
ds = list_datasets[0]

In [6]:
ds

<xarray.Dataset>
Dimensions:                       (sg_data_point: 1338, trajectory: 1,
                                   gps_info: 3, gc_event: 148)
Coordinates:
  * trajectory                    (trajectory) int32 503
    longitude                     (sg_data_point) float64 ...
    latitude                      (sg_data_point) float64 ...
    ctd_time                      (sg_data_point) datetime64[ns] ...
    ctd_depth                     (sg_data_point) float64 ...
Dimensions without coordinates: sg_data_point, gps_info, gc_event
Data variables: (12/226)
    surface_curr_north            float64 ...
    surface_curr_east             float64 ...
    start_of_climb_time           timedelta64[ns] ...
    sg_cal_volmax                 float64 ...
    sg_cal_vbd_min_cnts           int32 ...
    sg_cal_vbd_max_cnts           int32 ...
    ...                            ...
    conductivity                  (sg_data_point) float64 ...
    buoyancy                      (sg_data_point) float64 ...
    GPSE_qc                       |S1 ...
    GPS2_qc                       |S1 ...
    GPS1_qc                       |S1 ...
    CTD_qc                        |S1 ...
Attributes: (12/58)
    quality_control_version:         1.1
    base_station_micro_version:      3705
    time_coverage_resolution:        PT1S
    geospatial_vertical_max:         984.1416311048997
    sea_name:                        Labrador Sea
    mission:                         1
    ...                              ...
    disclaimer:                      Data has not been reviewed and is provid...
    geospatial_vertical_positive:    no
    date_created:                    2013-08-01T05:08:02Z
    geospatial_vertical_units:       meter
    dive_number:                     503
    history:                         Processing start:\n05:52:59 01 Aug 2013 ...

### Load datasets from a remote directory (URL)

In [7]:
# Specify the server where data are located
server = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/015/20040924/"

# Load and concatenate all datasets from the server, optionally specifying the range of profiles to load
list_datasets = readers.read_basestation(server, start_profile=500, end_profile=503)

## Convert to OG1 format

Process:

1. For one basestation dataset, split the dataset by dimension (`split_ds`)
3. Transform into OG1 format: dataset with dims `sg_data_point`
    - Change the dimension to `N_MEASUREMENTS`
    - Rename variables according to `vocabularies.standard_names` 
    - Assign variable attributes according to `vocabularies.vocab_attrs`.  (Note: This *could* go wrong since it makes assumptions about the input variables. May need additional handling.)
4. Add missing mandatory variables: 
    - From `split_ds[(gps_info,)]`, add the `LATITUDE_GPS`, `LONGITUDE_GPS` and `TIME_GPS` (Note: presently `TIME_GPS` is stripped before saving, but `TIME` values contain `TIME_GPS`)
    - Create `PROFILE_NUMBER` and `PHASE`
    - Calculate `DEPTH_Z` which is positive up
5. Update attributes for the file. 
    - Combines `creator` and `contributor` from original attributes into `contributor`
    - Adds `contributing_institutions` based on `institution`
    - Reformats time in `time_coverage_*` and `start_time`--> `start_date`
    - Adds `date_modified`
    - Renames `comments`-->`history`, `site`-->`summary`
    - Adds `title`, `platform`, `platform_vocabulary`, `featureType`, `Conventions`, `rtqc_method*` according to OceanGliders format
    - Retains `naming_authority`, `institution`, `project`, `geospatial_*` as OG attributes
    - Retains extra attributes: `license`, `keywords`, `keywords_vocabulary`, `file_version`, `acknowledgement`, `date_created`, `disclaimer`

Future behaviour to be added:

6. Retain the variables starting with `sg_cal` and check whether they vary over the mission (shouldn't)
6. Add sensors, using information in the `split_ds` with no dimensions
    - Need (from sg_cal_constants: `sg_cal` plus `volmax`, `vbd_cnts_per_cc`, `therm_expan`, `t_*`, `mass`, `hd_*`, `ctcor`, `cpcor`, `c_*`, `abs_compress`, `a`, `Tcor`, `Soc`, `Pcor`, `Foffset`)
    - Maybe also `reviewed`, `magnetic_variation` (which will change with position), `log_D_FLARE`, `flight_avg_speed_north` and `flight_avg_speed_east` also with `_gsm`, `depth_avg_curr_north` and `depth_avg_curr_east` also with `_gsm`, 
    `wlbb2f` - means sensor
    `sg_cal_mission_title`
    `sg_cal_id_str`
    `calibcomm_oxygen`
    `calibcomm`
    `sbe41` means ??
    `hdm_qc`
    `glider`
    
### Convert a single (sample) dataset

In [8]:
# Loads one dataset (p0150500_20050213.nc)
ds = readers.load_sample_dataset()

ds_OG1 = convertOG1.convert_to_OG1(ds)

# Check the results - uncomment the following lines to either generate a plot or show the variables.
plotters.plot_profile_depth(ds_OG1)

Variable 'vert_speed_gsm' not in OG1 vocabulary.
Variable 'time' not in OG1 vocabulary.
Variable 'speed_gsm' not in OG1 vocabulary.
Variable 'sound_velocity' not in OG1 vocabulary.
Variable 'north_displacement_gsm' not in OG1 vocabulary.
Variable 'longitude_gsm' not in OG1 vocabulary.
Variable 'latitude_gsm' not in OG1 vocabulary.
Variable 'horz_speed_gsm' not in OG1 vocabulary.
Variable 'glide_angle_gsm' not in OG1 vocabulary.
No conversion information found for cm s-1 to degrees
Variable 'eng_wlbb2f_VFtemp' not in OG1 vocabulary.
Variable 'eng_sbect_tempFreq' not in OG1 vocabulary.
Variable 'eng_sbect_condFreq' not in OG1 vocabulary.
Variable 'eng_elaps_t_0000' not in OG1 vocabulary.
Variable 'eng_elaps_t' not in OG1 vocabulary.
Variable 'eng_depth' not in OG1 vocabulary.
Variable 'east_displacement_gsm' not in OG1 vocabulary.
No conversion information found for micromoles/kg to percent
Variable 'depth' not in OG1 vocabulary.
Variable 'density' not in OG1 vocabulary.
Variable 'buoyan

sg_cal_calibcomm: SBE t12/c12 calibration 30DEC03
sg_cal_calibcomm_oxygen: SBE 43F s/n 041 calibration 22JAN04


TypeError: Input data must be a pandas DataFrame or xarray Dataset

In [ ]:
# Print to screen a table of the variables and variable attributes
#plotters.show_attributes(ds_OG1)
ds_OG1

### Convert mission from a local directory of basestation files

- For local data in the directory `input_dir`
- Creates a plot of ctd_depth against ctd_time.


In [ ]:
# Specify the input directory on your local machine
input_dir = "/Users/eddifying/Dropbox/data/sg015-ncei-download"

# Load and concatenate all datasets in the input directory
# Optionally, specify the range of profiles to load (start_profile, end_profile)
list_datasets = readers.read_basestation(input_dir, start_profile=500, end_profile=503)

# Convert the list of datasets to OG1
ds_OG1 = convertOG1.convert_to_OG1(list_datasets)

# Generate a simple plot
plotters.plot_profile_depth(ds_OG1)
#plotters.show_contents(ds_all,'attrs')

### Convert mission from the NCEI server (with p*nc files)

- Data from the sg015 mission in the Labrador Sea (https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.nodc:0111844), dataset identifier gov.noaa.nodc:0111844.


In [ ]:
# Specify the server where data are located
server = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/015/20040924/"

# Load and concatenate all datasets from the server, optionally specifying the range of profiles to load
list_datasets = readers.read_basestation(server, start_profile=500, end_profile=503)

# Convert the list of datasets to OG1
ds_OG1 = convertOG1.convert_to_OG1(list_datasets)

## Saving data

Due to problems with writing `xarray` datasets as netCDF when attributes are not of a specified type (`str`, `Number`, `np.ndarray`, `np.number`, `list`, `tuple`), a function was written `save_dataset`.

In [ ]:
# Write the file
# This writer catches errors in data types (DType errors) when using xr.to_netcdf()
# The solution is to convert them to strings, which may be undesired behaviour
output_file = os.path.join(data_path, 'demo_test.nc')
if os.path.exists(output_file):
    os.remove(output_file) 

writers.save_dataset(ds_OG1, output_file);

In [ ]:
# Load the data saved
ds1 = xr.open_dataset(output_file)

# Generate a simple plot
#plotters.show_contents(ds_all,'attrs')
plotters.plot_depth_colored(ds1, color_by='PROFILE_NUMBER')


## Run multiple missions

In [ ]:
# Add these to existing attributes - update to your details
contrib_to_append = vocabularies.contrib_to_append
print(contrib_to_append)

In [ ]:
# Specify a list of servers or local directories
input_locations = [
    # Either Iceland, Faroes or RAPID/MOCHA
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/005/20090829/", # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/005/20080606/", # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/005/20081106/", # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/012/20070831/", # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/014/20080214/",  # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/014/20080222/", # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20061112/",  # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20090605/", # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20071113/", # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20080607/",  # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/033/20100518/", # done  
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/033/20100903/", # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/101/20081108/",     # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/101/20061112/",    # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/101/20070609/",   # done
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/102/20061112/",  # done
    # Labrador Sea
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/015/20040924/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/014/20040924/",
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/008/20031002/",
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/004/20031002/",
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20050406/",
    # RAPID/MOCHA
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/033/20100729/",
    #"https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/034/20110128/",
]

for input_loc in input_locations:
    # Example usage
    ds_all = convertOG1.process_and_save_data(input_loc, output_dir=data_path, save=True,  run_quietly=True)